In [1]:
%run -i ../python/common.py
UC_SKIPTERMS=True
%run -i ../python/ln_preamble.py

# SLS Lecture 8 :  Writing some simple assembly programs



Spend some time writing some very simple assembly programs and learn to use the debugger so that we have enough skills to explore how things work.    We will be repeat various things in more detail in future lectures.

- Write `popcnt` in assemble code 
  - use gdb to play with the popcnt program
- Write a simple `add` in assembly code
  - use gdb to play with the add program
    - using the cpu as a glorified calculator 
      - first pass at CPU support for "numbers" 
- What happens if we let our programs continue
  - how do we successfully "halt/end" our execution 
    - `int3` trap 
       - tells OS to return control to debugger   
    - more generally how can we make a Kernel/System Call
  - revisit `add` programs adding exits  
    - `int3`
    - `exit` syscall
- Implicitly use our shell, editor, Make and Git knowledge to do the above

## Writing a `popcnt` assembly program

- Write a one instruction assembly program 
  1. first using .byte 
  2. using intel assembly instruction  
- Use gdb to explore how this instruction works
  - learn to use gdb to set register values
  - and how to execute and re-execute an instruction

In [2]:
display(Markdown(FileCodeBox(
    file="../src/popcnt_bb.S", 
    lang="gas", 
    title="<b>CODE: asm - The 'popcnt' assembly program",
    h="100%", 
    w="107em"
)))

<b>CODE: asm - The 'popcnt' assembly program
<div style="width:107em; height:100%; font-size:inherit; overflow: auto;" >


``` gas
	.intel_syntax noprefix
	
	.text
        
	.global _start 	
_start:
	popcnt rax, rbx   // same as  .byte 0xF3, 0x48, 0x0F, 0xB8, 0xD8  

```


</div>


In [3]:
display(Markdown('''
Here is a fully commented version of the same code.
'''))
display(Markdown(FileCodeBox(
    file="../src/popcnt.S", 
    lang="gas", 
    title="<b>CODE: asm - The commented 'popcnt' assembly program",
    h="100%", 
    w="107em"
)))


Here is a fully commented version of the same code.


<b>CODE: asm - The commented 'popcnt' assembly program
<div style="width:107em; height:100%; font-size:inherit; overflow: auto;" >


``` gas
/*  General antomy of a assembly program line
[lablel]:   <directive or opcode> [operands]  // comment
*/
	.intel_syntax noprefix   // assembler syntax to use <directive>
	                         // set assembly language format to intel  

	.text                    // linker section <directive>
	                         // let the linker know that what follows are cpu instructions to
	                         // to be executed -- uposed to values that represent data.
	                         // For historical reasons cpu instructions are called "text"
	                         
	.global _start           // linker symbol type <directive> 
	                         // makes the symbol _start in this case visible to the linker
	                         // The linker looks for an _start symbol so that it knows address
	                         // of the first instruction of our program
	
_start:                          // introduce a symbolic (human readable) label for "this" address
	                         // associates the address of this point in our program with the
      	                         // name following the ':' -- in our case _start
	                         // In our program or in the debugger we can use this name to
	                         // to refer to this location -- address.  And thus the values
	                         // that end up here.
//	.byte 0xF3, 0x48, 0x0F, 0xB8, 0xD8  // popcnt rax,rbx	
	popcnt rax, rbx          // ok the single intel opcode (instruction) that makes up
	                         // our program

        // Details about the assembler directives and general syntax that we will be using
        // https://sourceware.org/binutils/docs/as/
	// Intel instruction set reference -- documents the cpu memonics/instructions of
	//                                    the computer's processor that we are writing for
	// https://software.intel.com/content/www/us/en/develop/download/intel-64-and-ia-32-architectures-sdm-combined-volumes-2a-2b-2c-and-2d-instruction-set-reference-a-z.html
	

```


</div>


In [4]:
display(showET("Editor"))

<b>Edit</b>

We can use the `.byte` directive to set the values in memory to anything we like 
eg.

``` gas
     .byte 0xF3, 0x48, 0x0F, 0xB8, 0xD8  
```

But of course the real value is that we could have also simply written

``` gas
       popcnt rax, rbx          
```


In [5]:
display(showBT())

<b>Build</b>

In [6]:
display(Markdown(FileCodeBox(
    file="popcnt_build.sh", 
    lang="shell", 
#    title="<b>NOTES: on building popcnt", 
    h="100%", 
    w="100%")))


<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` shell
#To assemble and link the code we will use the following command:
gcc --static -g -nostartfiles -nostdlib -Wl,--build-id=none -Wa,-alh -Xlinker -Map=popcnt.map popcnt.S -o popcnt > popcnt.lst
# We can automate this using a makefile so that all we would need to do is:
make popcnt

```


</div>


In [7]:
display(showDT("Debugger"))

<b>Debug</b>

In [8]:
display(Markdown(FileCodeBox(
    file="popcnt_gdb.txt", 
    lang="shell", 
    title="", 
    h="100%", 
    w="100%")))


<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` shell
# To get the debugger going:
  gdb -tui popcnt
# Now we want use gdb command to poke around popcnt
# To setup the assembly syntax to intel: 
  set disassembly-flavor intel
# Configure a layout that is more friendly to assembly code:
  tui new-layout mylayout regs 3 {src 1 asm 1} 2 cmd 1 status 0
# Switch to the new layout:   
  layout mylayout
# force small command window to be sure lots of room for regs
  winh cmd 4
# make command window in focus so that arrows work as history  
  focus cmd
  
# Set a breakpoint at the start symbol so exection will stop their: 
  break _start
# Start the program running:
  run
# play around with popcnt

set rbx = 0b11
set rbx = 0xFFFE





```


</div>


## Writing an `add` assembly program

- re-enforce the steps to creating and debugging an assembly program
  - begin to explore CPU support for working with "numbers"
    - cpu as a calculator

- Lets work with the `add` instruction in a similar way that we did with `popcnt`
- explore the results of adding with binary, hex, unsigned and signed values
- explore overflow

- then make the program a little more complex:
``` gas
  movabs rbx, 0xdeadbeefdeadbeef
  mov    rax, 1
  add    rax, rbx
```

- lets use some more cool features of the intel instruction set
``` gas
  rdrand rbx                
  mov    rax, 1
  add    rax, rbx
  popcnt rbx, rax
```
- lets get a brief glimpse at how to use memory locations for the value
``` gas
        .intel_syntax noprefix
        .data
x:       .quad 142
y:       .quad 4200
sum:     .quad

        .text
        .global _start
_start:
        mov rax, QWORD PTR x
        add rax, QWORD PTR y
        mov QWORD PTR sum, rax
        int3
```
  
  
- try replacing add with `imul`, `and`, `or`, `xor`



In [9]:
display(showET("Editor"))

<b>Edit</b>

In [10]:
display(Markdown(FileCodeBox(
    file="../src/add.S", 
    lang="gas", 
    title="",
    h="100%", 
    w="100%"
)))


<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` gas
	.intel_syntax noprefix

	.text 	
	.global _start
_start:
	add rax, rbx

```


</div>


In [11]:
display(showBT())

<b>Build</b>

In [12]:
display(Markdown(FileCodeBox(
    file="add_build.sh", 
    lang="shell", 
#    title="<b>NOTES: on building add", 
    h="100%", 
    w="100%")))


<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` shell
gcc --static -g -nostartfiles -nostdlib -Wl,--build-id=none -Wa,-alh -Xlinker -Map=add.map add.S -o add > add.lst
# create makefile that automates
make add

```


</div>


In [13]:
display(showDT())

<b>Debug</b>

In [14]:
display(Markdown(FileCodeBox(
    file="add_gdb.txt", 
    lang="shell", 
    title="", 
    h="100%", 
    w="100%")))


<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` shell
gdb -tui add
set disassembly-flavor intel
tui new-layout mylayout regs 3 {src 1 asm 1} 2 cmd 1 status 0
layout mylayout
winh cmd 4
focus cmd
break _start
run

```


</div>


## Ending / Exiting our Program/Process

- What happens if we run our programs outside of the debugger?
  - why does this happen?

In [15]:
#display(showET())

In [16]:
#display(showBT())

In [17]:
display(showDT())

<b>Debug</b>

### How can we avoid this

1. TRAP: Use an instruction that tells the OS to 
    - stop the process and give control back to the debuggger
    - if no debugger is running just kill process and signal shell
        - Instruction: `int3`: 
        - Opcode: `0xCC` 
        - Description: `Interrupt 3 — trap to debugger`
2. Call OS Kernel Exit Process call
    - This is an example of calling an OS Kernel call to have the kernel do something for your process
    - We will look at this more but for the moment here is what is necessary to call `exit`
       - pass return value to Kernel 
       - exit/terminate process

### Interrupt 3 `int3` -- trap to debugger

<img src="/files/work/UndertheCovers/underthecovers/images/int3mp.png">

In [18]:
display(Markdown(FileCodeBox(
    file="../src/int3.S", 
    lang="gas", 
    title="",
    h="100%", 
    w="100%"
)))


<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` gas
	.intel_syntax noprefix

	.text 	
	.global _start
_start:
	int3
	

```


</div>


### Exit -- An OS service to terminate a process

To exit your process and return an exit value 
  - requires a call to the OS!

On Intel the instruction is `syscall`

<img src="/files/work/UndertheCovers/underthecovers/images/syscallmp.png">


### The OS System Calls

Each OS Kernel provides a set of calls that an a process can invoke using the `syscall` instruction on an Intel based computer

The Linux Kernel supports a very large number of system calls each is identified by a unique number that must be placed in `RAX` prior to executing the `syscall` instruction.  Additional arguments are passed in by setting other registers.  

With each version of the Kernel the table of calls changes.  Here is one site that provides a list



In [19]:
display(IFrame("https://filippo.io/linux-syscall-table/", height=600, width="100%"))

- From the above we can see that the `exit` system call number is `60`
- reading some man pages `man syscall` and `man syscalls` we find that
  - we must place `60` in `rax`
  - and that the value we want to return in `rdi`

In [20]:
display(Markdown(FileCodeBox(
    file="../src/exit_bb_bb.S", 
    lang="gas", 
    title="",
    h="100%", 
    w="100%"
)))


<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` gas
	.intel_syntax noprefix
	
	.text
	.global _start
_start:      	
	mov     rax, 60    # Linux exit system call number is 60       
	mov     rdi, 0     # rdi is return value 0 success

	syscall                         	

```


</div>


Operating system code usually provides files that you can include in your code so that you don't have to hardcode magic numbers like `60` for exit.  In Linux you can add the following file `#include <asm/unistd_64.h>` to get all the system call numbers.  You can then use `__NR_exit` to mean the number for the exit system call.

eg.

``` gas
    mov rax,__NR_exit # exit system call number
    mov rdi,0         # UNIX success value is 0
    syscall           # call OS. This will not return
```
    

#### Here is a fully documented fancy version

In [21]:
display(Markdown(
'''
A commented version that avoids "magic" numbers. 
'''    +
    
    FileCodeBox(
    file="../src/exit.S", 
    lang="gas", 
    title="",
    h="100%", 
    w="200%"
)))


A commented version that avoids "magic" numbers. 

<div style="width:200%; height:100%; font-size:inherit; overflow: auto;" >


``` gas
#include <asm/unistd_64.h>       // Pull in a file that contains all the OS system call numbers
				 // See discussion bellow to understand what is in this file
	
	.intel_syntax noprefix   // set assembly language format to intel

	// Define some constants so that we don't have magic numbers
	// in our code.  We use the .equ (equal) assembly directive
	// "This directive sets the value of symbol to expression."
	// (https://sourceware.org/binutils/docs/as/Equ.html#Equ)
	// FORMAT:	
	// .equ <SYMBOL>, <EXPRESSION>
	
	.equ UNIX_SUCCESS_VALUE, 0      
	.equ LINUX_SYSCALL_EXIT, __NR_exit

	.text                    // Place the following in the area that
	                         // instruction should be encoded and stored
	                         // for historical reasons it is called text
	.global _start
_start:                          // The linker knows to mark the _start address
                              	 // as location where execution should begin.
	                         // The OS will be sure to setup the CPU so that the
	                         // program counter is initialized with this address

	// To voluntarily hand control of the cpu back to the Operating system so it can
	// do somthing for us we use a special instruction
	//  -- on x86_64 this instruction is syscall (man syscall - table 1)
	//
	// Operating systems provide many functions that a program can call to
	// get things done.  eg.  open a stream, read or write bytes to or from a stream
	// or simply terminate and exit our program.
	//
	// Each call is identifed by a number we call a system call number.
	// On LINUX it expects us to load rax register with the system call number
	// prior to executing the syscall.  Once control is handed over to the OS
	// It will then look the number up and invoke the approriate function inside itself.
	// Other registers can be used to pass parmeters to the particular OS system call function
	// that we want to invoke.  The following is the conventions on X86_64 Linux
	// 
	//       Arch/ABI      arg1  arg2  arg3  arg4  arg5  arg6  arg7 
        // ────────────────────────────────────────────────────────────
        //       x86-64        rdi   rsi   rdx   r10   r8    r9    -
	// (see man syscall -- table 2)
	// The list of all Linux system call numbers can be found in a file so that programers
	// can know what number to use to invoke a particular call
	// On our installation of linux you can find it here
	//  /usr/include/x86_64-linux-gnu/asm/unistd_64.h
	// For an explanation of all the systems calls see man syscalls (note the s)
	// -- When we switch to understanding C we it will be easier to understand
	//    the above manual pages
	//
	// On LINUX 
	//   One of these calls is the EXIT system call that tells OS our program is done 
	//   (see man 2 exit)
	
	mov     rax, LINUX_SYSCALL_EXIT        // load rax with the Linux system call number for
	                                       // exit
	
	mov     rdi, UNIX_SUCCESS_VALUE        // load rdi with the exit status value we want
	                                       // to pass back to the program that launched us
	                                       // eg. the shell in which we ran our program from
	
	syscall                                // use the syscall instruction to hand control over
	                                       // to the OS.  Note since we will be terminated
	                                       // this instruction will be our last and nothing else
	                                       // need follow it
	

```


</div>
